# Simulated Annealing (Имитирано закаляване)

<div class="simulated-annealing-algorithm">
    <img src="images/simulated-annealing-algorithm.png" alt="Simulated Annealing" style="width: 700px; margin: 10px;"/>
</div>

### Параметри:
- **T, Tmin, Tmax** :: температурата е основен параметър, който определя поведението на алгоритъма
    - в началото, когато T е голямо, алгоритъмът прави разходка в ширина (random walk, exploration)
    - към края, когато T е малко, алгоритъмът експлоатира локален екстремум (hill climbing, обхождане в дълбочина, exploitation)
- **INIT()** - начална точка / начално решение.
- **NEIGHBOUR(T, best)** - дава следваща точка в зависимост от температурата и настоящата най-добра. Например точка от нормалното разпределение около точка **best** с дисперсия **Т**.
<div class="normal">
    <img src="images/normal.png" alt="Normal Distribution" style="width: 400px;"/>
</div>

- **ACCEPT(T, dE)** - определя с каква вероятност ще приемем по-лоша точка.
$$\displaystyle ACCEPT(T, \Delta E) = \mathrm{e}^{\frac{-\Delta E}{kT}}$$
- **ENERGY(X)** - целевата функция, която се оптимизира. При дискретните проблеми трябва да си измислим такава.
- **COOLING(T)** - правилото, по което се намалява температурата **T**
$$COOLING(T) = \alpha T \hspace{2pc} \alpha = 0.99$$

### Поведение:

- Explore $\displaystyle T \to \infty \hspace{1pc} \lim_{T \to \infty} \frac{-\Delta E}{kT} = 0 \hspace{2pc} \mathrm{e}^{0} = 1$

- Exploit $\displaystyle T \to 0 \hspace{1pc} \lim_{T \to 0} \frac{-\Delta E}{kT} = -\infty \hspace{2pc} \mathrm{e}^{-\infty} \to 0$

### Плюсове:
- лесно адаптиране на алгоритъма към даден проблем (непрекъснат или дискретен)
- не налага никакви ограничения върху целевата функция **ENERGY(X)**
    - няма нужда **ENERGY(X)** да е непрекъсната, може и да е **дискретна**
    - не изисква например да е гладка (да има N непрекъснати производни), производната да е Липшицова или Хесиана да е положително полу-дефинитна матрица, както при градиентните методи
    - няма ограничение за изпъкналост (единствен оптимум)
- паралелизъм / последователно пускане на алгоритъма там, където е приключил


### Минуси:
- евристичен, тоест не е гарантирано, че ще намерим оптимум
- не знаем колко близо сме до оптимума


In [2]:
import operator
import numpy as np
import math
from numpy import random

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

import matplotlib.animation as animation

%matplotlib nbagg

class SimulatedAnnealing():
    def __init__(self, minimize, T_min, T_max, init,
                 neighbour, energy, accept, cooling, should_log):
        self.compare = operator.lt if minimize else operator.gt
        self.T_min = T_min
        self.neighbour = neighbour
        self.energy = energy
        self.accept = accept
        self.cooling = cooling
        
        self.T = T_max
        self.best = init()
        self.step = 0
        self.should_log = should_log

    def iteration(self):
        next = self.neighbour(self.T, self.best)
        deltaE = self.energy(next) - self.energy(self.best)
        if self.compare(deltaE, 0):
            self.best = next
        elif random.random() < self.accept(self.T, deltaE):
            self.best = next

    def cool(self):
        self.T = self.cooling(self.T, self.best)

    def execute(self):
        while self.T > self.T_min:
            self.iteration()
            self.cool()
            if self.should_log:
                self.log()
            self.step += 1
        if self.should_log:
            print('\nf(' + str(self.best) + ') = ' + str(self.energy(self.best)))
        return self.best
    
    def log(self):
        if(self.step % 100 == 0):
            print(str(self.step) + " T="+str(self.T) +
                  "\tf(" + str(self.best) + ") = " + str(self.energy(self.best)))

# standart configuration parameters and functions for Simulated Annealing
t_min = 1e-5
t_max = 1

def init():
    return np.array([0.1, 1.4])

def neighbour(T, x):
    #return x + random.uniform(-T, T, x.shape)
    return random.normal(x, max(T, 0.1), x.shape)

def accept(T, deltaE, k=0.1):
    return math.exp(-(deltaE)/k/T)

def cooling(T, best):
    return T*0.99


# Knapsack 0-1 problem:

- Вход:
    - $C$ - capacity (капацитет на раницата)
    - $W = \{w_1, w_2, \dots, w_n\}$ - weights (тежест на предметите)
    - $V = \{v_1, v_2, \dots, v_n\}$ - values / prices (цени на предметите)
    - където $n = |W| = |V|$ - брой предмети

- Изход:
    - $X = \{x_1, x_2, \dots, x_n\}, x_i \in \{0, 1\}$ - характеристичен вектор на избраните предмети

- Задача:
    - maximize ${\displaystyle \sum _{i=1}^{n}v_{i}x_{i}}$
    - subject to ${\displaystyle \sum _{i=1}^{n}w_{i}x_{i}\leq C}$ and ${\displaystyle x_{i}\in \{0,1\}}$.

- NP-Complete: псевдополиномиално решение чрез динамично програмиране за $O(nC)$


In [3]:
# knapsack problem data set generator

DIM = 60
MAX_VALUE = 100000000
MAX_WEIGHT = 100000000
C = 1000000000
W = np.random.randint(MAX_WEIGHT, size=DIM)
V = np.random.randint(MAX_VALUE, size=DIM)
print(W)
print(V)

[96053538 16438579 92096382 46242468 62321143 99509620 66488384 11971134
 59172434  6128525 63796482 49674943 33824252 81192028 76596350 90245031
 75791848 49654722 89450609  4514730 95639921 33988890 50672803 90592095
 19806475 68025971 78586950 40967627 71709388 26340626 94292927 44157928
 51224097 96688972 98801516 36317266 83421681 26763858 71316501 56429031
 52773713 15071391  6087597 88703844 48648474 74449709 94120282 10909968
 25512517 41603398 85211837 77754917 32160665 82036654 85759970 19045682
 93213414  5263456 90341680 42846457]
[22954754 49490827 18677522 81496316 36590510 59747498 22170155 65697196
  8737847 73593188 43495251 13577334 58522129 62980501 46893544 84247212
 71549890 14740310 77119518  3211596 59401362   332336 90422511 62003048
 10302442 69942897 90122268 40308632 19715332 28192908 81526243  9446147
 17907610 17447100  7588196 38573393 95529071 29304579 20955992 15609112
 36840573 32652257 98609409 11026530 77885180 52873041 12050676 48242177
 47012551 615

In [4]:
from functools import partial


class KnapsackAnnealer:
    
    @staticmethod
    def init(DIM):
        return np.random.randint(2, size=DIM)  # DIM zeros and ones

    
    # naive choice of next knapsack
    @staticmethod
    def neighbour(DIM, cost_fn, T, knapsack):
        new_knapsack = np.copy(knapsack)
        new_knapsack[random.randint(DIM)] ^= 1  # flip the bit
        while cost_fn(new_knapsack) == 0:
            new_knapsack[random.randint(DIM)] ^= 1
        return new_knapsack

    
    @staticmethod
    def cost(W, V, C, knapsack):
        total_weight = sum([W[i] for i, chosen in enumerate(knapsack) if chosen])
        total_value = sum([V[i] for i, chosen in enumerate(knapsack) if chosen])
        return 0 if total_weight > C else total_value

    
    @staticmethod
    def accept(T, deltaE, k=0.1):
        return math.exp(deltaE / k / T)

    
    def __init__(self, weights, values, capacity):
        self.W = weights
        self.V = values
        self.C = capacity
        self.DIM = len(self.W)
        
        init = partial(KnapsackAnnealer.init, self.DIM)
        self.cost = partial(KnapsackAnnealer.cost, self.W, self.V, self.C)
        neighbour = partial(KnapsackAnnealer.neighbour, self.DIM, self.cost)
        
        self.annealer = SimulatedAnnealing(False, t_min, t_max, init, neighbour, self.cost,
                                           KnapsackAnnealer.accept, cooling, False)

        
    def run(self):
        return self.annealer.execute()

In [5]:
# maximization of the 0-1 Knapsack Problem with data taken from
# https://people.sc.fsu.edu/~jburkardt/datasets/knapsack_01/knapsack_01.html

# DIM = 15
# W = [70, 73, 77, 80, 82, 87, 90, 94, 98, 106, 110, 113, 115, 118, 120]
# V = [135, 139, 149, 150, 156, 163, 173, 184, 192, 201, 210, 214, 221, 229, 240]
# C = 750
# OPTIMUM = 1458
# OPTIMAL_KNAPSACK = [1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1]

W = [382745, 799601, 909247, 729069, 467902, 44328, 34610, 698150,
     823460, 903959, 853665, 551830, 610856, 670702, 488960, 951111,
     323046, 446298, 931161, 31385, 496951, 264724, 224916, 169684]
V = [825594, 1677009, 1676628, 1523970, 943972, 97426, 69666, 1296457,
     1679693, 1902996, 1844992, 1049289, 1252836, 1319836, 953277,
     2067538, 675367, 853655, 1826027, 65731, 901489, 577243, 466257, 369261]
C = 6404180

annealer = KnapsackAnnealer(W, V, C)
solution = annealer.run()
annealer.cost(solution)

OPTIMUM = 13549094
OPTIMAL_KNAPSACK = [1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1]

mismatches = 0
for i, item in enumerate(solution):
    mismatches += 1 if solution[i] != OPTIMAL_KNAPSACK[i] else 0
print(str(mismatches) + " items in the selection mismatch with the optimal solution")
print("{0:.3f}% less optimal then the most optimal solution".format((1 - annealer.cost(solution)/OPTIMUM) * 100))

4 items in the selection mismatch with the optimal solution
0.614% less optimal then the most optimal solution


In [6]:
# error analysis
from numpy import mean
from functools import reduce
from operator import eq, add
from pprint import pprint


def ints(filename):
    return [int(line) for line in open(filename)]


def data(id, type):
    return ints("data/p0{}_{}.txt".format(id, type))


test_data = [{
    "weights": data(i, 'w'),
    "prices": data(i, 'p'),
    "capacity": data(i, 'c')[0],
    "optimum": data(i, 's')
 } for i in range(1, 9)]


def test(data):
    annealer = KnapsackAnnealer(data['weights'], data['prices'], data['capacity'])
    approximation = annealer.run()
    deviation = (1 - annealer.cost(approximation) / annealer.cost(data['optimum'])) * 100
    mismatches = count_mismatches(list(approximation), data['optimum'])
    
    return (deviation, mismatches)


def count_mismatches(list1, list2):
    return len(list1) - reduce(add, map(eq, list1, list2), 0)

test_results = [[test(data) for _ in range(100)] for data in test_data]
average_errors = [tuple(map(mean, zip(*errors))) for errors in test_results]

# print(test_results)

pprint([{
    'errors': {
        'deviation': problem[1][0],
        'mismatches': problem[1][1]        
    },
    'dimensions': len(problem[0]['weights']),
    'capacity': problem[0]['capacity'],
    'maxWeight': max(problem[0]['weights']),
    'maxPrice': max(problem[0]['prices'])
 } for problem in zip(test_data, average_errors)])


[{'capacity': 165,
  'dimensions': 10,
  'errors': {'deviation': 0.0, 'mismatches': 0.0},
  'maxPrice': 92,
  'maxWeight': 89},
 {'capacity': 26,
  'dimensions': 5,
  'errors': {'deviation': 0.0, 'mismatches': 0.0},
  'maxPrice': 24,
  'maxWeight': 12},
 {'capacity': 190,
  'dimensions': 6,
  'errors': {'deviation': 0.0, 'mismatches': 0.0},
  'maxPrice': 64,
  'maxWeight': 80},
 {'capacity': 50,
  'dimensions': 7,
  'errors': {'deviation': 0.0, 'mismatches': 0.0},
  'maxPrice': 70,
  'maxWeight': 31},
 {'capacity': 104,
  'dimensions': 8,
  'errors': {'deviation': 0.0, 'mismatches': 0.0},
  'maxPrice': 450,
  'maxWeight': 45},
 {'capacity': 170,
  'dimensions': 7,
  'errors': {'deviation': 0.0, 'mismatches': 0.0},
  'maxPrice': 617,
  'maxWeight': 60},
 {'capacity': 750,
  'dimensions': 15,
  'errors': {'deviation': 0.33264746227709246,
             'mismatches': 4.0099999999999998},
  'maxPrice': 240,
  'maxWeight': 120},
 {'capacity': 6404180,
  'dimensions': 24,
  'errors': {'deviat

In [12]:
import time
 
def totalvalue(comb):
    ' Totalise a particular combination of items'
    totwt = totval = 0
    for item, wt, val in comb:
        totwt  += wt
        totval += val
    return (totval, -totwt) if totwt <= 400 else (0, 0)


def knapsack01_dp(items, limit):
    table = [[0 for w in range(limit + 1)] for j in range(len(items) + 1)]
 
    for j in range(1, len(items) + 1):
        item, wt, val = items[j-1]
        for w in range(1, limit + 1):
            if wt > w:
                table[j][w] = table[j-1][w]
            else:
                table[j][w] = max(table[j-1][w],
                                  table[j-1][w-wt] + val)
 
    result = []
    w = limit
    for j in range(len(items), 0, -1):
        was_added = table[j][w] != table[j-1][w]
 
        if was_added:
            item, wt, val = items[j-1]
            result.append(items[j-1])
            w -= wt
 
    return result


d = test_data[4]
print(d)

print(reduce(add, map(operator.mul, d['optimum'], d['prices']), 0))

items = (
    ("map", 9, 150), ("compass", 13, 35), ("water", 153, 200), ("sandwich", 50, 160),
    ("glucose", 15, 60), ("tin", 68, 45), ("banana", 27, 60), ("apple", 39, 40),
    ("cheese", 23, 30), ("beer", 52, 10), ("suntan cream", 11, 70), ("camera", 32, 30),
    ("t-shirt", 24, 15), ("trousers", 48, 10), ("umbrella", 73, 40),
    ("waterproof trousers", 42, 70), ("waterproof overclothes", 43, 75),
    ("note-case", 22, 80), ("sunglasses", 7, 20), ("towel", 18, 12),
    ("socks", 4, 50), ("book", 30, 10),
    )

items = list(zip([str(i) for i in range(len(d['weights']))], d['weights'], d['prices']))
print(items)

start_time = time.time()

bagged = knapsack01_dp(items, d['capacity'])
print("Bagged the following items\n  " +
      '\n  '.join(sorted(item for item,_,_ in bagged)))
val, wt = totalvalue(bagged)
print("for a total value of %i and a total weight of %i" % (val, -wt))
print("total time:", time.time() - start_time)

{'optimum': [1, 0, 1, 1, 1, 0, 1, 1], 'prices': [350, 400, 450, 20, 70, 8, 5, 5], 'capacity': 104, 'weights': [25, 35, 45, 5, 25, 3, 2, 2]}
900
[('0', 25, 350), ('1', 35, 400), ('2', 45, 450), ('3', 5, 20), ('4', 25, 70), ('5', 3, 8), ('6', 2, 5), ('7', 2, 5)]
Bagged the following items
  0
  2
  3
  4
  6
  7
for a total value of 900 and a total weight of 104
total time: 0.0006682872772216797
